### Import libraries

In [1]:
from IPython.core.display import display
from src.utils.preprocessing import classic_preprocessing
from src.utils.get_data import import_data, split_experts
from src.utils.train import hyperparameter_tuning_cv
from src.utils.config import *
import pandas as pd
import numpy as np

In [2]:
%load_ext autoreload
%autoreload 2

### Load Data

In [3]:
DATA_PATH = '../../../data'
X_fine, y_fine = import_data(DATA_PATH, segmentation_type='fine',
                             drop_user_features=True,
                             drop_expert=True)

In [4]:
# For each expert separately
X_e, y_e = import_data(DATA_PATH, segmentation_type='fine',
                       drop_user_features=True,
                       drop_expert=False)

In [5]:
display(X_fine.head())
display(y_fine.head())

EEPD50_100  EEPD100_150  \
subject                              file_id                            
008ba489-31ad-44d8-856b-fcf72369dc46 0               6.0          6.0   
                                     1               3.0          3.0   
                                     2               4.0          4.0   
                                     3               4.0          4.0   
                                     4               4.0          4.0   

                                              EEPD150_200  EEPD200_250  \
subject                              file_id                             
008ba489-31ad-44d8-856b-fcf72369dc46 0                5.0          4.0   
                                     1                3.0          4.0   
                                     2                3.0          4.0   
                                     3                4.0          4.0   
                                     4                5.0          4.0   

                                              EEPD250_300  EEPD300_350  \
subject                              file_id                             
008ba489-31ad-44d8-856b-fcf72369dc46 0                5.0          5.0   
                                     1                4.0          5.0   
                                     2                3.0          3.0   
                                     3                4.0          5.0   
                                     4                4.0          5.0   

                                              EEPD350_400  EEPD400_450  \
subject                              file_id                             
008ba489-31ad-44d8-856b-fcf72369dc46 0                6.0          6.0   
                                     1                4.0          4.0   
                                     2                4.0          4.0   
                                     3                3.0          4.0   
                                     4                3.0          4.0   

                                              EEPD450_500  EEPD500_550  ...  \
subject                              file_id                            ...   
008ba489-31ad-44d8-856b-fcf72369dc46 0                5.0          6.0  ...   
                                     1                4.0          3.0  ...   
                                     2                4.0          4.0  ...   
                                     3                4.0          3.0  ...   
                                     4                4.0          3.0  ...   

                                              MFCC_std11  MFCC_std12  \
subject                              file_id                           
008ba489-31ad-44d8-856b-fcf72369dc46 0         12.109140   11.702864   
                                     1         10.000873    8.967152   
                                     2          8.318832    8.688411   
                                     3         13.294838    8.694027   
                                     4         12.814457    8.501382   

                                              Crest_Factor  Cough_Length  \
subject                              file_id                               
008ba489-31ad-44d8-856b-fcf72369dc46 0            7.649371      0.767250   
                                     1            8.001334      0.568917   
                                     2           11.189755      0.581667   
                                     3            7.840153      0.540250   
                                     4            7.869070      0.546333   

                                              PSD_225-425  PSD_450-550  \
subject                              file_id                             
008ba489-31ad-44d8-856b-fcf72369dc46 0           0.398299     0.026407   
                                     1           0.230052     0.086242   
                                     2           0.105747     0.180344   
               

Label
subject                              file_id       
008ba489-31ad-44d8-856b-fcf72369dc46 0          1.0
                                     1          1.0
                                     2          1.0
                                     3          1.0
                                     4          1.0

### Preprocessing

In [6]:
X_fine = classic_preprocessing(X_fine, stop=None, dummy=False)

In [7]:
# Save the expert feature for split
expert = X_e['Expert'].copy()
X_e = classic_preprocessing(X_e, stop=None, dummy=False)
X_e['Expert'] = expert.values

### Split expert models

In [8]:
X_e_1, y_e_1, X_e_2, y_e_2, X_e_3, y_e_3 = split_experts(X_e, y_e)

### Grid search

#### 1. Logistic regression

In [9]:
log_results = hyperparameter_tuning_cv(model='logistic', data=X_fine, labels=y_fine.Label, cv_k=5,
                                       params=LOGISTIC_PARAMS)

display(log_results)

f1_score  roc_auc_score  accuracy_score
max_iter oversampling                                         
10000    True          0.424358       0.579228        0.583072
         False         0.116406       0.522802        0.733619
100000   True          0.424358       0.579228        0.583072
         False         0.116406       0.522802        0.733619

In [10]:
# Best model parameters

best_log = log_results.iloc[[log_results.reset_index()['roc_auc_score'].idxmax()]]

display(best_log)

,,f1_score,roc_auc_score,accuracy_score
max_iter,oversampling,,,
10000,True,0.424358,0.579228,0.583072


In [11]:
# for each expert
log_results_1 = hyperparameter_tuning_cv(model='logistic', data=X_e_1, labels=y_e_1, cv_k=5,
                                         params=LOGISTIC_PARAMS)
log_results_2 = hyperparameter_tuning_cv(model='logistic', data=X_e_2, labels=y_e_2, cv_k=5,
                                         params=LOGISTIC_PARAMS)
log_results_3 = hyperparameter_tuning_cv(model='logistic', data=X_e_3, labels=y_e_3, cv_k=5,
                                         params=LOGISTIC_PARAMS)

In [12]:
# Best model parameters

best_log_1 = log_results_1.iloc[[
    log_results_1.reset_index()['roc_auc_score'].idxmax()]]
best_log_2 = log_results_2.iloc[[
    log_results_2.reset_index()['roc_auc_score'].idxmax()]]
best_log_3 = log_results_3.iloc[[
    log_results_3.reset_index()['roc_auc_score'].idxmax()]]

display(best_log_1, best_log_2, best_log_3)

,,f1_score,roc_auc_score,accuracy_score
max_iter,oversampling,,,
10000,True,0.365471,0.655199,0.69919


,,f1_score,roc_auc_score,accuracy_score
max_iter,oversampling,,,
10000,True,0.326309,0.571407,0.614173


,,f1_score,roc_auc_score,accuracy_score
max_iter,oversampling,,,
10000,False,0.611063,0.644425,0.647724


#### 2. Linear Discriminant Analysis

In [13]:
lda_results = hyperparameter_tuning_cv(model='lda', data=X_fine, labels=y_fine.Label, cv_k=5,
                                       params=LDA_PARAMS)

display(lda_results)

,f1_score,roc_auc_score,accuracy_score
oversampling,,,
True,0.427548,0.581468,0.582882
False,0.122605,0.524261,0.733904


In [14]:
# Best model parameters

best_lda = lda_results.iloc[[lda_results.reset_index()['roc_auc_score'].idxmax()]]

display(best_lda)

,f1_score,roc_auc_score,accuracy_score
oversampling,,,
True,0.427548,0.581468,0.582882


In [15]:
# For each expert
lda_results_1 = hyperparameter_tuning_cv(model='lda', data=X_e_1, labels=y_e_1, cv_k=5,
                                         params=LDA_PARAMS)
lda_results_2 = hyperparameter_tuning_cv(model='lda', data=X_e_2, labels=y_e_2, cv_k=5,
                                         params=LDA_PARAMS)
lda_results_3 = hyperparameter_tuning_cv(model='lda', data=X_e_3, labels=y_e_3, cv_k=5,
                                         params=LDA_PARAMS)

In [16]:
best_lda_1 = lda_results_1.iloc[[
    lda_results_1.reset_index()['roc_auc_score'].idxmax()]]
best_lda_2 = lda_results_2.iloc[[
    lda_results_2.reset_index()['roc_auc_score'].idxmax()]]
best_lda_3 = lda_results_3.iloc[[
    lda_results_3.reset_index()['roc_auc_score'].idxmax()]]

display(best_lda_1, best_lda_2, best_lda_3)

,f1_score,roc_auc_score,accuracy_score
oversampling,,,
True,0.37692,0.669532,0.692707


,f1_score,roc_auc_score,accuracy_score
oversampling,,,
True,0.326972,0.571416,0.607874


,f1_score,roc_auc_score,accuracy_score
oversampling,,,
False,0.610512,0.645963,0.649655


#### 3. K-nearest Neighbors

In [17]:
knn_results = hyperparameter_tuning_cv(model='knn', data=X_fine, labels=y_fine.Label, cv_k=5,
                                       params=KNN_PARAMS)

display(knn_results)

f1_score  roc_auc_score  accuracy_score
n_neighbors oversampling                                         
1           True          0.499423       0.646364        0.621113
            False         0.468613       0.637330        0.719734
2           True          0.492222       0.645516        0.668379
            False         0.276739       0.566098        0.742748
3           True          0.489337       0.629823        0.557775
            False         0.405214       0.606831        0.728293
4           True          0.485425       0.630811        0.594864
            False         0.267270       0.562644        0.741227
5           True          0.479726       0.614988        0.519829
            False         0.355189       0.588455        0.734665
6           True          0.479520       0.619738        0.553971
            False         0.240226       0.555529        0.741797
7           True          0.468249       0.598287        0.490062
            False         0.304540       0.570866        0.734475
8           True          0.471853       0.607622        0.524013
            False         0.208849       0.546089        0.738849
9           True          0.461607       0.587486        0.468759
            False         0.264146       0.559017        0.735521
10          True          0.466193       0.598067        0.501094
            False         0.189225       0.541795        0.739515
11          True          0.457860       0.580934        0.455445
            False         0.240340       0.553373        0.737233
12          True          0.460167       0.588289        0.481598
            False         0.173324       0.537501        0.738374
13          True          0.452522       0.572274        0.440323
            False         0.217504       0.547510        0.737423
14          True          0.455422       0.580389        0.465716
            False         0.160209       0.535268        0.739515
15          True          0.449291       0.566752        0.430433
            False         0.196911       0.542955        0.738374

In [18]:
# Best model parameters

best_knn = knn_results.iloc[[knn_results.reset_index()['roc_auc_score'].idxmax()]]

display(best_knn)

,,f1_score,roc_auc_score,accuracy_score
n_neighbors,oversampling,,,
1,True,0.499423,0.646364,0.621113


In [19]:
knn_results_1 = hyperparameter_tuning_cv(
    model='knn', data=X_e_1, labels=y_e_1, cv_k=5, params=KNN_PARAMS)
knn_results_2 = hyperparameter_tuning_cv(
    model='knn', data=X_e_2, labels=y_e_2, cv_k=5, params=KNN_PARAMS)
knn_results_3 = hyperparameter_tuning_cv(
    model='knn', data=X_e_3, labels=y_e_3, cv_k=5, params=KNN_PARAMS)

In [20]:
best_knn_1 = knn_results_1.iloc[[
    knn_results_1.reset_index()['roc_auc_score'].idxmax()]]
best_knn_2 = knn_results_2.iloc[[
    knn_results_2.reset_index()['roc_auc_score'].idxmax()]]
best_knn_3 = knn_results_3.iloc[[
    knn_results_3.reset_index()['roc_auc_score'].idxmax()]]

display(best_knn_1, best_knn_2, best_knn_3)

,,f1_score,roc_auc_score,accuracy_score
n_neighbors,oversampling,,,
1,True,0.429214,0.729216,0.703079


,,f1_score,roc_auc_score,accuracy_score
n_neighbors,oversampling,,,
1,True,0.434726,0.674297,0.674803


,,f1_score,roc_auc_score,accuracy_score
n_neighbors,oversampling,,,
1,True,0.689127,0.703541,0.703724


#### 4. Support Vector Classifier

In [21]:
svc_results = hyperparameter_tuning_cv(model='svc', data=X_fine, labels=y_fine.Label, cv_k=5,
                                       params=SVC_PARAMS)

display(svc_results)

f1_score  roc_auc_score  accuracy_score
kernel gamma oversampling                                         
linear 0.10  True          0.428619       0.579581        0.571945
             False         0.000000       0.500000        0.730480
rbf    0.10  True          0.084634       0.519190        0.737803
             False         0.029801       0.507509        0.734475
linear 0.01  True          0.428619       0.579581        0.571945
             False         0.000000       0.500000        0.730480
rbf    0.01  True          0.434135       0.602052        0.648407
             False         0.050792       0.510803        0.734094

In [22]:
# Best model parameters

best_svc = svc_results.iloc[[svc_results.reset_index()['roc_auc_score'].idxmax()]]

display(best_svc)

,,,f1_score,roc_auc_score,accuracy_score
kernel,gamma,oversampling,,,
rbf,0.01,True,0.434135,0.602052,0.648407


In [23]:
svc_results_1 = hyperparameter_tuning_cv(
    model='svc', data=X_e_1, labels=y_e_1, cv_k=5, params=SVC_PARAMS)
svc_results_2 = hyperparameter_tuning_cv(
    model='svc', data=X_e_2, labels=y_e_2, cv_k=5, params=SVC_PARAMS)
svc_results_3 = hyperparameter_tuning_cv(
    model='svc', data=X_e_3, labels=y_e_3, cv_k=5, params=SVC_PARAMS)

In [24]:
best_svc_1 = svc_results_1.iloc[[
    svc_results_1.reset_index()['roc_auc_score'].idxmax()]]
best_svc_2 = svc_results_2.iloc[[
    svc_results_2.reset_index()['roc_auc_score'].idxmax()]]
best_svc_3 = svc_results_3.iloc[[
    svc_results_3.reset_index()['roc_auc_score'].idxmax()]]


display(best_svc_1, best_svc_2, best_svc_3)

,,,f1_score,roc_auc_score,accuracy_score
kernel,gamma,oversampling,,,
rbf,0.01,True,0.414728,0.67027,0.8


,,,f1_score,roc_auc_score,accuracy_score
kernel,gamma,oversampling,,,
linear,0.1,True,0.329036,0.571256,0.586877


,,,f1_score,roc_auc_score,accuracy_score
kernel,gamma,oversampling,,,
rbf,0.01,False,0.639119,0.675562,0.679448


#### 5. Naive Bayes Classifier

In [25]:
nb_results = hyperparameter_tuning_cv(model='naive_bayes', data=X_fine, labels=y_fine.Label, cv_k=5,
                                      params=NAIVE_BAYES_PARAMS)

display(nb_results)

,f1_score,roc_auc_score,accuracy_score
oversampling,,,
True,0.373249,0.558792,0.624156
False,0.335659,0.558044,0.674560


In [26]:
# Best model parameters

best_nb = nb_results.iloc[[nb_results.reset_index()['roc_auc_score'].idxmax()]]

display(best_nb)

,f1_score,roc_auc_score,accuracy_score
oversampling,,,
True,0.373249,0.558792,0.624156


In [27]:
nb_results_1 = hyperparameter_tuning_cv(
    model='naive_bayes', data=X_e_1, labels=y_e_1, cv_k=5, params=NAIVE_BAYES_PARAMS)
nb_results_2 = hyperparameter_tuning_cv(
    model='naive_bayes', data=X_e_2, labels=y_e_2, cv_k=5, params=NAIVE_BAYES_PARAMS)
nb_results_3 = hyperparameter_tuning_cv(
    model='naive_bayes', data=X_e_3, labels=y_e_3, cv_k=5, params=NAIVE_BAYES_PARAMS)

In [28]:
best_nb_1 = nb_results_1.iloc[[
    nb_results_1.reset_index()['roc_auc_score'].idxmax()]]
best_nb_2 = nb_results_2.iloc[[
    nb_results_2.reset_index()['roc_auc_score'].idxmax()]]
best_nb_3 = nb_results_3.iloc[[
    nb_results_3.reset_index()['roc_auc_score'].idxmax()]]

display(best_nb_1, best_nb_2, best_nb_3)

,f1_score,roc_auc_score,accuracy_score
oversampling,,,
True,0.330198,0.626599,0.645381


,f1_score,roc_auc_score,accuracy_score
oversampling,,,
True,0.291533,0.53156,0.554068


,f1_score,roc_auc_score,accuracy_score
oversampling,,,
True,0.467552,0.588102,0.601655


#### 6. Decision Tree

In [29]:
dt_results = hyperparameter_tuning_cv(model='decision_tree', data=X_fine, labels=y_fine.Label, cv_k=5,
                                      params=DECISION_TREE_PARAMS)

display(dt_results)

f1_score  roc_auc_score  accuracy_score
max_depth oversampling                                         
3         True          0.407376       0.548584        0.515264
          False         0.090122       0.512900        0.725915
5         True          0.340146       0.545595        0.625773
          False         0.113746       0.517697        0.726201
7         True          0.321666       0.540204        0.642796
          False         0.206697       0.531217        0.710984

In [30]:
# Best model parameters

best_dt = dt_results.iloc[[dt_results.reset_index()['roc_auc_score'].idxmax()]]

display(best_dt)

,,f1_score,roc_auc_score,accuracy_score
max_depth,oversampling,,,
3,True,0.407376,0.548584,0.515264


In [31]:
dt_results_1 = hyperparameter_tuning_cv(
    model='decision_tree', data=X_e_1, labels=y_e_1, cv_k=5, params=DECISION_TREE_PARAMS)
dt_results_2 = hyperparameter_tuning_cv(
    model='decision_tree', data=X_e_2, labels=y_e_2, cv_k=5, params=DECISION_TREE_PARAMS)
dt_results_3 = hyperparameter_tuning_cv(
    model='decision_tree', data=X_e_3, labels=y_e_3, cv_k=5, params=DECISION_TREE_PARAMS)

In [32]:
best_dt_1 = dt_results_1.iloc[[
    dt_results_1.reset_index()['roc_auc_score'].idxmax()]]
best_dt_2 = dt_results_2.iloc[[
    dt_results_2.reset_index()['roc_auc_score'].idxmax()]]
best_dt_3 = dt_results_3.iloc[[
    dt_results_3.reset_index()['roc_auc_score'].idxmax()]]

display(best_dt_1, best_dt_2, best_dt_3)

,,f1_score,roc_auc_score,accuracy_score
max_depth,oversampling,,,
7,True,0.33903,0.628576,0.705997


,,f1_score,roc_auc_score,accuracy_score
max_depth,oversampling,,,
7,True,0.278989,0.541102,0.64042


,,f1_score,roc_auc_score,accuracy_score
max_depth,oversampling,,,
3,True,0.539619,0.600186,0.606897


#### 7. Random Forest

In [33]:
rf_results = hyperparameter_tuning_cv(model='random_forest', data=X_fine, labels=y_fine.Label, cv_k=5,
                                      params=RANDOM_FOREST_PARAMS)

display(rf_results)

f1_score  roc_auc_score  accuracy_score
max_depth n_estimators oversampling                                         
3         3            True          0.377711       0.545160        0.574703
                       False         0.028024       0.502972        0.728103
          5            True          0.390667       0.556167        0.584593
                       False         0.030939       0.504937        0.730290
          7            True          0.387177       0.554017        0.584593
                       False         0.023614       0.503744        0.730385
5         3            True          0.359833       0.550120        0.618735
                       False         0.100785       0.514546        0.725345
          5            True          0.377097       0.557309        0.613885
                       False         0.072420       0.512603        0.730956
          7            True          0.383379       0.566733        0.630718
                       False         0.053271       0.509597        0.731526
7         3            True          0.365454       0.552389        0.619116
                       False         0.150343       0.522784        0.721446
          5            True          0.347968       0.552130        0.643272
                       False         0.119606       0.521192        0.729815
          7            True          0.352150       0.556221        0.648883
                       False         0.082440       0.513735        0.729815

In [34]:
# Best model parameters

best_rf = rf_results.iloc[[rf_results.reset_index()['roc_auc_score'].idxmax()]]

display(best_rf)

,,,f1_score,roc_auc_score,accuracy_score
max_depth,n_estimators,oversampling,,,
5,7,True,0.383379,0.566733,0.630718


In [35]:
rf_results_1 = hyperparameter_tuning_cv(
    model='random_forest', data=X_e_1, labels=y_e_1, cv_k=5, params=RANDOM_FOREST_PARAMS)
rf_results_2 = hyperparameter_tuning_cv(
    model='random_forest', data=X_e_2, labels=y_e_2, cv_k=5, params=RANDOM_FOREST_PARAMS)
rf_results_3 = hyperparameter_tuning_cv(
    model='random_forest', data=X_e_3, labels=y_e_3, cv_k=5, params=RANDOM_FOREST_PARAMS)

In [36]:
best_rf_1 = rf_results_1.iloc[[
    rf_results_1.reset_index()['roc_auc_score'].idxmax()]]
best_rf_2 = rf_results_2.iloc[[
    rf_results_2.reset_index()['roc_auc_score'].idxmax()]]
best_rf_3 = rf_results_3.iloc[[
    rf_results_3.reset_index()['roc_auc_score'].idxmax()]]

display(best_rf_1, best_rf_2, best_rf_3)

,,,f1_score,roc_auc_score,accuracy_score
max_depth,n_estimators,oversampling,,,
3,7,True,0.355743,0.64702,0.692058


,,,f1_score,roc_auc_score,accuracy_score
max_depth,n_estimators,oversampling,,,
5,7,True,0.282745,0.546703,0.669291


,,,f1_score,roc_auc_score,accuracy_score
max_depth,n_estimators,oversampling,,,
7,7,True,0.581136,0.613384,0.616276


#### 8. Gradient Boosting

In [37]:
gb_results = hyperparameter_tuning_cv(model='gradient_boosting', data=X_fine, labels=y_fine.Label, cv_k=5,
                                      params=GRADIENT_BOOSTING_PARAMS)

display(gb_results)

f1_score  roc_auc_score  accuracy_score
max_depth n_estimators oversampling                                         
3         3            True          0.353651       0.552765        0.630052
                       False         0.000000       0.500000        0.730480
          5            True          0.356908       0.555510        0.634237
                       False         0.001408       0.500289        0.730575
          7            True          0.364988       0.559862        0.641274
                       False         0.004207       0.500862        0.730766
5         3            True          0.322786       0.549411        0.664384
                       False         0.001423       0.500227        0.730480
          5            True          0.308005       0.549489        0.678554
                       False         0.009757       0.501940        0.731051
          7            True          0.316451       0.554360        0.682834
                       False         0.018786       0.503734        0.731526
7         3            True          0.312629       0.548094        0.673229
                       False         0.001426       0.500163        0.730385
          5            True          0.300993       0.549696        0.688065
                       False         0.021883       0.503718        0.730766
          7            True          0.289003       0.550665        0.700048
                       False         0.043470       0.508996        0.733238

In [38]:
# Best model parameters

best_gb = gb_results.iloc[[gb_results.reset_index()['roc_auc_score'].idxmax()]]

display(best_gb)

,,,f1_score,roc_auc_score,accuracy_score
max_depth,n_estimators,oversampling,,,
3,7,True,0.364988,0.559862,0.641274


In [39]:
gb_results_1 = hyperparameter_tuning_cv(
    model='gradient_boosting', data=X_e_1, labels=y_e_1, cv_k=5, params=GRADIENT_BOOSTING_PARAMS)
gb_results_2 = hyperparameter_tuning_cv(
    model='gradient_boosting', data=X_e_2, labels=y_e_2, cv_k=5, params=GRADIENT_BOOSTING_PARAMS)
gb_results_3 = hyperparameter_tuning_cv(
    model='gradient_boosting', data=X_e_3, labels=y_e_3, cv_k=5, params=GRADIENT_BOOSTING_PARAMS)

In [40]:
best_gb_1 = gb_results_1.iloc[[
    gb_results_1.reset_index()['roc_auc_score'].idxmax()]]
best_gb_2 = gb_results_2.iloc[[
    gb_results_2.reset_index()['roc_auc_score'].idxmax()]]
best_gb_3 = gb_results_3.iloc[[
    gb_results_3.reset_index()['roc_auc_score'].idxmax()]]
display(best_gb_1, best_gb_2, best_gb_3)

,,,f1_score,roc_auc_score,accuracy_score
max_depth,n_estimators,oversampling,,,
7,3,True,0.361354,0.644428,0.733874


,,,f1_score,roc_auc_score,accuracy_score
max_depth,n_estimators,oversampling,,,
3,7,True,0.30588,0.56181,0.661155


,,,f1_score,roc_auc_score,accuracy_score
max_depth,n_estimators,oversampling,,,
7,7,True,0.611766,0.639416,0.641931


### Results

In [41]:
#display('logistic', best_log)
#display('lda', best_lda)
#display('knn', best_knn)
#display('svc', best_svc)
#display('naive_bayes', best_nb)
#display('decision_tree', best_dt)
#display('random_forest', best_rf)
#display('gradient_boosting', best_gb)

In [42]:
weighted_average = (np.array([len(X_e_1), len(X_e_2), len(X_e_3)]) / len(X_e))

e_log = np.sum(pd.concat([best_log_1, best_log_2, best_log_3]
                         ).roc_auc_score.values * weighted_average)
e_lda = np.sum(pd.concat([best_lda_1, best_lda_2, best_lda_3]
                         ).roc_auc_score.values * weighted_average)
e_knn = np.sum(pd.concat([best_knn_1, best_knn_2, best_knn_3]
                         ).roc_auc_score.values * weighted_average)
e_svc = np.sum(pd.concat([best_svc_1, best_svc_2, best_svc_3]
                         ).roc_auc_score.values * weighted_average)
e_nb = np.sum(pd.concat([best_nb_1, best_nb_2, best_nb_3]
                        ).roc_auc_score.values * weighted_average)
e_dt = np.sum(pd.concat([best_dt_1, best_dt_2, best_dt_3]
                        ).roc_auc_score.values * weighted_average)
e_rf = np.sum(pd.concat([best_rf_1, best_rf_2, best_rf_3]
                        ).roc_auc_score.values * weighted_average)
e_gb = np.sum(pd.concat([best_gb_1, best_gb_2, best_gb_3]
                        ).roc_auc_score.values * weighted_average)

In [43]:
# Results expert features
# First row is Expert 1, second Expert 2, third Expert 3
#display('logistic', pd.concat([best_log_1,best_log_2,best_log_3]))
#display('lda', pd.concat([best_lda_1,best_lda_2,best_lda_3]))
display('knn', pd.concat([best_knn_1, best_knn_2, best_knn_3]))
#display('svc', pd.concat([best_svc_1,best_svc_2,best_svc_3]))
#display('naive_bayes', pd.concat([best_nb_1,best_nb_2,best_nb_3]))
#display('decision_tree', pd.concat([best_dt_1,best_dt_2,best_dt_3]))
#display('random_forest', pd.concat([best_rf_1,best_rf_2,best_rf_3]))
#display('gradient_boosting', pd.concat([best_gb_1,best_gb_2,best_gb_3]))

'knn'

f1_score  roc_auc_score  accuracy_score
n_neighbors oversampling                                         
1           True          0.429214       0.729216        0.703079
            True          0.434726       0.674297        0.674803
            True          0.689127       0.703541        0.703724

In [44]:
results = pd.DataFrame(data={'models': ['logistic', 'lda', 'knn', 'svc', 'naive_bayes', 'decision_tree',
                                        'random_forest', 'gradient_boosting'],
                             'auc_best': [best_log.roc_auc_score.values[0], best_lda.roc_auc_score.values[0],
                                          best_knn.roc_auc_score.values[0], best_svc.roc_auc_score.values[0],
                                          best_nb.roc_auc_score.values[0], best_dt.roc_auc_score.values[0],
                                          best_rf.roc_auc_score.values[0], best_gb.roc_auc_score.values[0]],
                             'auc_expert_weighted': [e_log, e_lda, e_knn, e_svc, e_nb, e_dt, e_rf, e_gb]})

display(results)

,models,auc_best,auc_expert_weighted
0,logistic,0.579228,0.621144
1,lda,0.581468,0.625879
2,knn,0.646364,0.700479
3,svc,0.602052,0.636241
4,naive_bayes,0.558792,0.578915
5,decision_tree,0.548584,0.587115
6,random_forest,0.566733,0.599100
7,gradient_boosting,0.559862,0.612784


### Conclusions

In [45]:
# Save dataframe
# results.to_pickle("results_fine_no_metadata.pkl")

In [46]:
results.mean(axis=0)

auc_best               0.580385
auc_expert_weighted    0.620207
dtype: float64

In [47]:
results.loc[results['auc_best'].argmax()]

models                      knn
auc_best               0.646364
auc_expert_weighted    0.700479
Name: 2, dtype: object

In [48]:
results.loc[results['auc_expert_weighted'].argmax()]

models                      knn
auc_best               0.646364
auc_expert_weighted    0.700479
Name: 2, dtype: object